In [1]:
## conda install -c conda-forge dlib
import numpy as np
import argparse
import cv2
import dlib
import imutils
import matplotlib.pyplot as plt

In [2]:
path_to_shape_predictor = "models/shape_predictor_68_face_landmarks.dat"
image_path = "data/sample_image.jpg"

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(path_to_shape_predictor)

In [5]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    
    return (x, y, w, h)

In [6]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    return coords

In [17]:
def extract_features(keypoints):
    assert keypoints.shape == (68,2)
    
    extr_points = []
    feats = []
    
    extr_points.append(keypoints[0])
    extr_points.append(keypoints[16])
    
    # width head
    denom = np.linalg.norm(keypoints[0]-keypoints[16])
    
    # adding points for visualization
    extr_points.append(keypoints[36])
    extr_points.append(keypoints[39])
    extr_points.append(keypoints[42])
    extr_points.append(keypoints[45])
    extr_points.append(keypoints[27])
    extr_points.append(keypoints[8])
    extr_points.append(keypoints[48])
    extr_points.append(keypoints[54])
    extr_points.append(keypoints[31])
    extr_points.append(keypoints[35])
    extr_points.append(keypoints[33])
    extr_points.append(keypoints[51])
    extr_points.append(keypoints[57])
    
    # eyes
    dist_eyes = np.linalg.norm(keypoints[39]-keypoints[42])
    width_eye_l = np.linalg.norm(keypoints[36]-keypoints[39])
    width_eye_r = np.linalg.norm(keypoints[42]-keypoints[45])
    dist_eye_l = np.linalg.norm(keypoints[36]-keypoints[27])
    dist_eye_r = np.linalg.norm(keypoints[27]-keypoints[45])
    
    feats.append(dist_eyes/denom)
    feats.append(width_eye_l/denom)
    feats.append(width_eye_r/denom)
    feats.append(dist_eye_l/denom)
    feats.append(dist_eye_r/denom)
    
    #head
    len_head = np.linalg.norm(keypoints[27]-keypoints[8])
    len_nose = np.linalg.norm(keypoints[27]-keypoints[33])
    width_nose = np.linalg.norm(keypoints[31]-keypoints[35])
    dist_nose_lips = np.linalg.norm(keypoints[33]-keypoints[51])
    dist_lips_chin = np.linalg.norm(keypoints[57]-keypoints[8])
    width_lips = np.linalg.norm(keypoints[48]-keypoints[54])
    
    feats.append(len_head/denom)
    feats.append(len_nose/denom)
    feats.append(width_nose/denom)
    feats.append(dist_nose_lips/denom)
    feats.append(dist_lips_chin/denom)
    feats.append(width_lips/denom)
    
    print(len(feats))
    
    return extr_points, feats

In [23]:
# pipeline to extract features from the face

# load the input image, resize it, and convert it to grayscale
image = cv2.imread(image_path)
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)
shape = predictor(gray, rect)
shape = shape_to_np(shape)

keypoints, feats = extract_features(shape)
#print(rects[0])

11


In [19]:


# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = shape_to_np(shape)
    
    keypoints, feats = extract_features(shape)

    # convert dlib's rectangle to a OpenCV-style bounding box
    # [i.e., (x, y, w, h)], then draw the face bounding box
    (x, y, w, h) = rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # show the face number
    # cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
        #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # loop over the (x, y)-coordinates for the facial landmarks
    # and draw them on the image
    for (x, y) in keypoints:
        cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
        
# show the output image with the face detections + facial landmarks
# plt.imshow(image)
cv2.imshow("Output", image)
cv2.waitKey(0)

11


-1